# My First Neural Network - Diabetes Risk Prediction
As a pretty commom example to start studying Deep Learning and Neural Networks, this is a try of using the well-known India Pima Dataset, which contains some health data like weight, age and number of times women got pregnant, and also a boolean data containing whether the person has diabetes or not. The network goal is then to predict the risk of a random person to develop diabetes based on the other inputs. 

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import models
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
np.random.seed(7)

We use this np.random.seed piece so that our "random numbers" are always the same random numbers, which allows for different runs on the code starting from the same numbers, since they are first generated randomly. 
The dataset is loaded in a .csv file called "Dados_Teste2_Projeto10.csv", and also stored in the same fold as the program file. 
We divide this dataset in two parts: training and testing. This is done by the function train_test_split. We used a test_size of 0.5, which means that half of the samples will be used for training and half for testing. The random_state piece has the same function as the random.seed. Since the dividing is done randomly, it's nice that the random samples for training and testing are always the same, so that we can adjust the other parameters without having another thing to worry.

In [5]:
#Loading External dataset - Indian Pima Diabetes
data = np.loadtxt("Dados_Teste2_Projeto10.csv", delimiter = ",")

#Spliting into X (inputs) and Y (outputs)
x = data[:,0:8]
y = data[:,8]

#Spliting into train an test samples
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.5, random_state=5)

Now we can build our neural network, which as hard as it is to understand its working principles and fully get what's going on, is extremely easy to build using the Keras library. There are some key points that should have some attention:
- Since our output is binary (1 - Yes, 0 - No), it would be nice if our network could return some number between 0 and 1 corresponding to the probability of the person to present diabetes.
- Careful on the number of neurons and layers to be used. Too many layers and neurons might seem great since our error would be minimum, but would cause overfitting, which makes our network pretty much useless.


To take care of the first point, we use as activation function of our output layer one called Sigmoid. Not getting to technical, this function takes any real input and map it into a value between 0 and 1, which can therefore be read as a probabiliy. The other functions, here represented by "relu" are not thaaat important, and might be different ones. The best choice is the one that gives the highest accuracy (which here happens to be the relu). 
All our layers will be Dense, which means that every neuron from one layer is connected to every neuron in the next layer. The number of neurons is pretty much arbitrary, just like the number of layers; and there is no formal consense on which architerure is better, every case is a case. Here we use a very standart and small number for both to adress our not-so-big dataset.

In [7]:
#Creating the model
model = Sequential()

#Input and Hidden Layers
model.add(Dense(8, activation='relu',
                           input_shape=(x_train.shape[1],)))
model.add(Dense(32, input_dim=8, activation = 'relu')) 
model.add(Dense(64, activation = 'relu'))

#Output Layer
model.add(Dense(1, activation = 'sigmoid'))

Now that we have created the architeture of our network, it's time to define how the network is gonna operate. This includes the choice of a loss function, an optimizer and some metrics to evaluate the model. Since we are computing something that has some "probabily" meaning as the output, a great loss function for it is the "Binary Crossentropy". As fancy as the name is, it's just a way of computing the difference between two probability distributions, instead of an usual square error. Actually, it's a bit more complicated, but as a casual programer, this is enough to understand when to use it. For the optimizer, the choice of Adam is kinda standart, since it's a very useful and versatile optimizer, but each problem might have one special optimizer that best fit the regression; the full concept of these optimizers is extremely deep and can get as difficult as wanted, but the main idea of them all is the same: based on the loss value, it uses the derivatives on each neuron to adjust the weight values iteratively. As Metric, accuracy is the choice here since it shows a number also between 0 and 1 that repreents how reliable our model is in predicting a probability. 

For the actual fitting part, we chose 1000 epochs, which represents how many times our program will go through all data to perform it's adjustments. We also chose a batch size of 10. While the number of epochs is directly proportional to the model performance, the batch size is not always easy to choose. In general, higher values of it means a slower training, but lead to a more stable model in higher epochs, whereas small values mean a quick learning, but might generate instability on higher epochs. 
The "verbose" command defines how much information we want to see in the screen while the network is trained. When equals 0, it means "silent mode", so the only it shows is a small warning that the fitting is finished. When equals to 1, it will show all loss and metrics values for each epoch, which is nice because we can see how our network is improving with the epochs, but sometimes makes the screen too messy.

In [9]:
#Adjusting operational parameters
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ['accuracy'])

#Training the model
model.fit(x_train, y_train, epochs = 1000, batch_size=10, validation_data = (x_test, y_test), verbose = 0)

After the fitting is complete, we can now evaluate how well our model has performed. Keras already has a function to do this, called "evaluate", for obvious reasons. We then just call it and see what result we get.

In [14]:
#evaluating the model (accuracy)
scores = model.evaluate(x,y)
print("\n%s: %.2f%%" % (model.metrics_names[1], 100*scores[1]))

768/768 [==============================] - 0s 16us/step

acc: 82.42%


So here we have it, 82% of accuracy. If it's a low or high value, we should then take a better look on the problem it's trying to solve. If one person is diagonsed with 82% of diabetes risk, this person should probably start to be more cautious, and it's extremely useful for health care systems. It's not a completely reliable and decisive tool, it's just a way of trying to extract more information from data. 

Finally, we can now simulate how our model would behave with a new sample. Without getting too deep in what the inputs actually are (is pretty easy to find on internet, just search for India Pima Diabetes), we will create a fictitious sample and run it. This fictitious person has no diabetes, so we should expect a low value output for the model.

In [16]:
x_new = np.array([[2,85,64,23,92,27.1,0.171,17]])
y_new = model.predict(x_new)
print("X=%s, Y=%s" % (x_new[0], y_new[0]))

X=[ 2.    85.    64.    23.    92.    27.1    0.171 17.   ], Y=[3.3284576e-11]


Our output is pratically zero, so this person should probably not care so much about diabetes at all! Let's now see how it works with a person who do have diabetes.

In [19]:
x_new = np.array([[6,148,72,35,0,33.6,0.627,50]])
y_new = model.predict(x_new)
print("X=%s, Y=%s" % (x_new[0], y_new[0]))

X=[  6.    148.     72.     35.      0.     33.6     0.627  50.   ], Y=[0.9119154]


As expected, more than 91% of chance of presenting diabetes, so it's a very likely candidate who do have to worry about developing diabetes. 
And that's how a basic neural network works. This is a very simple example, but also very powerful and show how actually easy it is to create neural networks. Opposing to the hard maths and dark magic behind it, using and creating it is surprisingly simple, and some great results might be achieved. 